# Evaluate model
Eavluate the output of the model using PESQ and STOI metrics.



In [ ]:
!conda install pandas
!conda install librosa

In [1]:
#Import basics and check everything works
import tensorflow as tf
from tensorflow import keras

print("Versions:", tf.version.VERSION, tf.version.GIT_VERSION)
print("GPU availablilty:", tf.test.is_gpu_available())
print("Eager execution:", tf.executing_eagerly())

#Quick test
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))

Versions: 1.14.0 unknown
GPU availablilty: True
Eager execution: False
hello, Tensor("MatMul:0", shape=(1, 1), dtype=float32)


In [2]:
#Add tools dir to path
import os
lib_path = os.path.join(os.getcwd(), 'lib')
print("lib:", lib_path)

pesq_path = os.path.join(lib_path, 'pesq.exe')
print('PESQ:', pesq_path)

import sys
sys.path.append(lib_path)

lib: d:\Code\External\SDA\lib
PESQ: d:\Code\External\SDA\lib\pesq.exe


In [4]:
from scipy.io import wavfile
import numpy as np
import pathlib
import os
import shutil
import pandas as pd
import librosa

import IPython.display as ipd
import matplotlib.pyplot as plt
#%matplotlib inline

BASE_DIR = os.path.join(os.getcwd(), "data", "wavs")
print(BASE_DIR)
train_dir = os.path.join(BASE_DIR, "train")
print(train_dir)
test_dir = os.path.join(BASE_DIR, "test")
print(test_dir)
model_dir = os.path.join(os.getcwd(), "model")
print(model_dir)
results_base_dir = os.path.join(os.getcwd(), "results")

model_name = "model-100"
print("model:", model_name)

results_dir = os.path.join(results_base_dir, os.path.splitext(model_name)[0])
os.makedirs(results_dir, exist_ok=True)
print(results_dir)

ModuleNotFoundError: No module named 'librosa'

In [8]:
def get_pesq_mos(pesq_path, clean_wav, degraded_wav, verbose=True):
    """
    Compare degraded_wav to clean_wav using PESQ.
    Returns float MOS score in range -0.5-4.5 (?!)
    """
    
    if verbose: 
        print(f'Command: {pesq_path} +16000 {clean_wav} {degraded_wav}')
    pesq_output = !{pesq_path} +16000 {clean_wav} {degraded_wav}
    results = pesq_output[-1]
    if verbose:
        print("Raw results:", results)

    mos = float(results.split('=')[1].split('\t')[0].strip())
    if verbose:
        print("MOS:", mos)
    
    return mos

In [11]:
#Load model
model_path = os.path.join(model_dir, model_name)
print("Model path:", f"{model_path}.json")
ae_model = keras.models.model_from_json(f"{model_path}.json")
ae_model.load_weights(f"{model_path}.h5")

Model path: d:\Code\External\SDA\model\model-100.json


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Load test set

In [6]:
#Get files
print(test_dir)
test_noisy_mel_files = list(pathlib.Path(test_dir).glob("*-N-mel.npy"))
num_noisy_mel = len(test_noisy_mel_files)

test_clean_mel_files = list(pathlib.Path(test_dir).glob("*-C-mel.npy"))
num_clean_mel = len(test_clean_mel_files)

test_noisy_wav_files = list(pathlib.Path(test_dir).glob("*-N.wav"))
num_noisy_wav = len(test_noisy_wav_files)

test_clean_wav_files = list(pathlib.Path(test_dir).glob("*-C.wav"))
num_clean_wav = len(test_clean_wav_files)

assert (num_noisy_mel == num_clean_mel) and (num_noisy_wav == num_clean_wav)

num_test_files = num_noisy_mel
print("Loaded", num_test_files, "clean and noisy wavs and mels")

D:\Code\Denoising\E1-Autoencoder\data\wavs\test
Loaded 290 clean and noisy wavs and mels


In [11]:
#Load test data

#Be explicit about the Wav <-> Mel conversion settings
wmc_settings = {
    "fft_size": 2048,
    "fft_step": 64,
    "spec_threshold": 4,
    "mel_n_freq_components": 100,
    "mel_start_freq": 300,
    "mel_end_freq": 8000,
    "mel_shorten_factor": 10,
    "griffin_lim_iterations": 10,
    "internal_sample_rate": 48000
}
mel_converter = wmc.WavMelConverter()
mel_converter.settings = wmc_settings

original_sr = 16000
original_duration = 1.0 #seconds

base_names = []

for idx in range(num_test_files):
    noisy_mel_file = test_noisy_mel_files[idx]
    
    #Get name to check the rest against
    base_name = os.path.basename(noisy_mel_file)[:-len("-N-mel.npy")]
    print(f"{idx+1}/{num_test_files}: Base name: {base_name}")
    base_names.append(base_name)
    
    #print("Noisy mel:", noisy_mel_file)
    clean_mel_file = test_clean_mel_files[idx]
    #print("Clean mel:", clean_mel_file)
    assert os.path.basename(clean_mel_file).startswith(base_name)
    
    noisy_wav_file = test_noisy_wav_files[idx]
    #print("Noisy wav:", noisy_wav_file)
    assert os.path.basename(noisy_wav_file).startswith(base_name)

    clean_wav_file = test_clean_wav_files[idx]
    #print("Clean wav:", clean_wav_file)
    assert os.path.basename(clean_wav_file).startswith(base_name)
    
    #Skip files that already exist
    if os.path.isfile(os.path.join(results_dir, f"{base_name}-C.wav")) and \
        os.path.isfile(os.path.join(results_dir, f"{base_name}-N.wav")) and \
        os.path.isfile(os.path.join(results_dir, f"{base_name}-C-mel.wav")) and \
        os.path.isfile(os.path.join(results_dir, f"{base_name}-N-mel.wav")) and \
        os.path.isfile(os.path.join(results_dir, f"{base_name}-D-mel.wav")):
        print(base_name, "processed, skipping.")
        continue
    
    #Copy clean as baseline 
    shutil.copyfile(clean_wav_file, os.path.join(results_dir, os.path.basename(clean_wav_file)))
    
    #do WAV -> MEL -> WAV to get max improvement
    mel_converter.load_wav_from_file(clean_wav_file)
    clean_mel = mel_converter.mel_spectrogram
    mel_converter.load_mel_data(clean_mel, original_sr, original_duration)
    mel_converter.save_wav_to_file(os.path.join(results_dir, f"{base_name}-C-mel.wav"))
    
    #Copy noisy
    shutil.copyfile(noisy_wav_file, os.path.join(results_dir, os.path.basename(noisy_wav_file)))
    
    #Load noisy mel, convert it and save
    print("Loading noisy mel:", noisy_mel_file)
    mel_converter.load_mel_from_file(noisy_mel_file, original_sr, original_duration)
    print("Saving wav from noisy mel...")
    mel_converter.save_wav_to_file(os.path.join(results_dir, f"{base_name}-N-mel.wav"))
    
    #Denoise using model
    print("Denoising...")
    batch_of_one = np.array([mel_converter.mel_spectrogram])
    denoised_mel_batch = ae_model.predict(batch_of_one)
    
    #Save denoised wav
    denoised_wav_path = os.path.join(results_dir, f"{base_name}-D-mel.wav")
    print("Save denoised wav to:", denoised_wav_path)
    mel_converter.load_mel_data(denoised_mel_batch[0], original_sr, original_duration)
    mel_converter.save_wav_to_file(denoised_wav_path)

print("*** Done ***")

1/290: Base name: 0_en-AU-Wavenet-A_0_10-0
0_en-AU-Wavenet-A_0_10-0 processed, skipping.
2/290: Base name: 0_en-AU-Wavenet-A_0_10-1
0_en-AU-Wavenet-A_0_10-1 processed, skipping.
3/290: Base name: 0_en-AU-Wavenet-A_0_10-2
0_en-AU-Wavenet-A_0_10-2 processed, skipping.
4/290: Base name: 0_en-AU-Wavenet-A_0_10-3
0_en-AU-Wavenet-A_0_10-3 processed, skipping.
5/290: Base name: 0_en-AU-Wavenet-A_0_10-4
0_en-AU-Wavenet-A_0_10-4 processed, skipping.
6/290: Base name: 0_en-AU-Wavenet-A_0_10-5
0_en-AU-Wavenet-A_0_10-5 processed, skipping.
7/290: Base name: 0_en-AU-Wavenet-A_0_12-0
0_en-AU-Wavenet-A_0_12-0 processed, skipping.
8/290: Base name: 0_en-AU-Wavenet-A_0_12-1
0_en-AU-Wavenet-A_0_12-1 processed, skipping.
9/290: Base name: 0_en-AU-Wavenet-A_0_12-2
0_en-AU-Wavenet-A_0_12-2 processed, skipping.
10/290: Base name: 0_en-AU-Wavenet-A_0_12-3
0_en-AU-Wavenet-A_0_12-3 processed, skipping.
11/290: Base name: 0_en-AU-Wavenet-A_0_12-4
0_en-AU-Wavenet-A_0_12-4 processed, skipping.
12/290: Base name: 

11_en-AU-Wavenet-D_0_12-0 processed, skipping.
112/290: Base name: 11_en-AU-Wavenet-D_0_12-1
11_en-AU-Wavenet-D_0_12-1 processed, skipping.
113/290: Base name: 11_en-AU-Wavenet-D_0_12-2
11_en-AU-Wavenet-D_0_12-2 processed, skipping.
114/290: Base name: 11_en-AU-Wavenet-D_0_12-3
11_en-AU-Wavenet-D_0_12-3 processed, skipping.
115/290: Base name: 11_en-AU-Wavenet-D_0_12-4
11_en-AU-Wavenet-D_0_12-4 processed, skipping.
116/290: Base name: 11_en-AU-Wavenet-D_0_14-0
11_en-AU-Wavenet-D_0_14-0 processed, skipping.
117/290: Base name: 11_en-AU-Wavenet-D_0_14-1
11_en-AU-Wavenet-D_0_14-1 processed, skipping.
118/290: Base name: 11_en-AU-Wavenet-D_0_14-2
11_en-AU-Wavenet-D_0_14-2 processed, skipping.
119/290: Base name: 11_en-AU-Wavenet-D_0_14-3
11_en-AU-Wavenet-D_0_14-3 processed, skipping.
120/290: Base name: 1_en-AU-Wavenet-A_0_10-0
1_en-AU-Wavenet-A_0_10-0 processed, skipping.
121/290: Base name: 1_en-AU-Wavenet-A_0_10-1
1_en-AU-Wavenet-A_0_10-1 processed, skipping.
122/290: Base name: 1_en-AU

c:\users\benhe\.conda\envs\homl2\lib\site-packages\scipy\ndimage\interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\4_en-AU-Wavenet-C_0_12-1-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\4_en-AU-Wavenet-C_0_12-1-D-mel.wav
202/290: Base name: 4_en-AU-Wavenet-C_0_12-2


c:\users\benhe\.conda\envs\homl2\lib\site-packages\scipy\ndimage\interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\4_en-AU-Wavenet-C_0_12-2-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\4_en-AU-Wavenet-C_0_12-2-D-mel.wav
203/290: Base name: 4_en-AU-Wavenet-C_0_14-0
Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\4_en-AU-Wavenet-C_0_14-0-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\4_en-AU-Wavenet-C_0_14-0-D-mel.wav
204/290: Base name: 4_en-AU-Wavenet-C_0_14-1
Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\4_en-AU-Wavenet-C_0_14-1-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\4_en-AU-Wavenet-C_0_14-1-D-mel.wav
205/290: Base name: 4_en-AU-Wavenet-C_0_14-2
Loading noisy mel: D:\Code\Denoising\E1-Autoencod

228/290: Base name: 5_en-AU-Wavenet-A_0_14-1
Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\5_en-AU-Wavenet-A_0_14-1-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\5_en-AU-Wavenet-A_0_14-1-D-mel.wav
229/290: Base name: 5_en-AU-Wavenet-A_0_14-2
Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\5_en-AU-Wavenet-A_0_14-2-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\5_en-AU-Wavenet-A_0_14-2-D-mel.wav
230/290: Base name: 5_en-AU-Wavenet-A_0_14-3
Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\5_en-AU-Wavenet-A_0_14-3-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\5_en-AU-Wavenet-A_0_14-3-D-mel.wav
231/290: Base name: 5_en-AU-Wavenet-B_0_10-0
Load

254/290: Base name: 5_en-AU-Wavenet-C_0_14-0
Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\5_en-AU-Wavenet-C_0_14-0-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\5_en-AU-Wavenet-C_0_14-0-D-mel.wav
255/290: Base name: 5_en-AU-Wavenet-C_0_14-1
Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\5_en-AU-Wavenet-C_0_14-1-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\5_en-AU-Wavenet-C_0_14-1-D-mel.wav
256/290: Base name: 5_en-AU-Wavenet-C_0_14-2
Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\5_en-AU-Wavenet-C_0_14-2-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\5_en-AU-Wavenet-C_0_14-2-D-mel.wav
257/290: Base name: 5_en-AU-Wavenet-C_0_14-3
Load

280/290: Base name: 9_en-AU-Wavenet-B_0_12-1
Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\9_en-AU-Wavenet-B_0_12-1-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\9_en-AU-Wavenet-B_0_12-1-D-mel.wav
281/290: Base name: 9_en-AU-Wavenet-B_0_14-0
Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\9_en-AU-Wavenet-B_0_14-0-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\9_en-AU-Wavenet-B_0_14-0-D-mel.wav
282/290: Base name: 9_en-AU-Wavenet-C_0_10-0
Loading noisy mel: D:\Code\Denoising\E1-Autoencoder\data\wavs\test\9_en-AU-Wavenet-C_0_10-0-N-mel.npy
Saving wav from noisy mel...
Denoising...
Save denoised wav to: D:\Code\Denoising\E1-Autoencoder\results\ae_model_offline_rev_mel_1000\9_en-AU-Wavenet-C_0_10-0-D-mel.wav
283/290: Base name: 9_en-AU-Wavenet-C_0_10-1
Load

In [13]:
#Pesq test

results = {}
for i, base_name in enumerate(base_names):
    print(f"{i+1}/{num_test_files}. Calculating PESQ scores for: {base_name}")
    clean = os.path.join(results_dir, f"{base_name}-C.wav")
    noisy = os.path.join(results_dir, f"{base_name}-N.wav")
    clean_mel = os.path.join(results_dir, f"{base_name}-C-mel.wav")
    noisy_mel = os.path.join(results_dir, f"{base_name}-N-mel.wav")
    denoised = os.path.join(results_dir, f"{base_name}-D-mel.wav")
    
    #Compare to base clean wav
    mel_mos_loss = get_pesq_mos(pesq_path, clean, clean_mel, verbose=False)
    noisy_mos_loss = get_pesq_mos(pesq_path, clean, noisy, verbose=False)
    noise_mos_loss = get_pesq_mos(pesq_path, clean, noisy_mel, verbose=False)
    denoised_mos = get_pesq_mos(pesq_path, clean, denoised, verbose=False)
    
    #Compare to mel processed clean wav
    clean_mel_noise_mos_loss = get_pesq_mos(pesq_path, clean_mel, noisy_mel, verbose=False)
    clean_mel_denoised_mos = get_pesq_mos(pesq_path, clean_mel, denoised, verbose=False)
    
    
    results[base_name] = [mel_mos_loss, noisy_mos_loss, noise_mos_loss, denoised_mos, clean_mel_noise_mos_loss, clean_mel_denoised_mos]

0/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_10-0
1/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_10-1
2/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_10-2
3/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_10-3
4/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_10-4
5/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_10-5
6/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_12-0
7/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_12-1
8/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_12-2
9/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_12-3
10/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_12-4
11/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_14-0
12/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_14-1
13/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_14-2
14/290. Calculating PESQ scores for: 0_en-AU-Wavenet-A_0_14-3
15/290. Calculating PESQ scores for: 0_en-AU-Wavenet-B_0_10-0
16/290. Calculatin

131/290. Calculating PESQ scores for: 1_en-AU-Wavenet-A_0_14-1
132/290. Calculating PESQ scores for: 1_en-AU-Wavenet-A_0_14-2
133/290. Calculating PESQ scores for: 1_en-AU-Wavenet-A_0_14-3
134/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_10-0
135/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_10-1
136/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_10-2
137/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_10-3
138/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_10-4
139/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_12-0
140/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_12-1
141/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_12-2
142/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_12-3
143/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_14-0
144/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_14-1
145/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B_0_14-2
146/290. Calculating PESQ scores for: 1_en-AU-Wavenet-B

262/290. Calculating PESQ scores for: 5_en-AU-Wavenet-D_0_12-0
263/290. Calculating PESQ scores for: 5_en-AU-Wavenet-D_0_12-1
264/290. Calculating PESQ scores for: 5_en-AU-Wavenet-D_0_12-2
265/290. Calculating PESQ scores for: 5_en-AU-Wavenet-D_0_12-3
266/290. Calculating PESQ scores for: 5_en-AU-Wavenet-D_0_14-0
267/290. Calculating PESQ scores for: 5_en-AU-Wavenet-D_0_14-1
268/290. Calculating PESQ scores for: 5_en-AU-Wavenet-D_0_14-2
269/290. Calculating PESQ scores for: 5_en-AU-Wavenet-D_0_14-3
270/290. Calculating PESQ scores for: 9_en-AU-Wavenet-A_0_10-0
271/290. Calculating PESQ scores for: 9_en-AU-Wavenet-A_0_10-1
272/290. Calculating PESQ scores for: 9_en-AU-Wavenet-A_0_12-0
273/290. Calculating PESQ scores for: 9_en-AU-Wavenet-A_0_12-1
274/290. Calculating PESQ scores for: 9_en-AU-Wavenet-A_0_14-0
275/290. Calculating PESQ scores for: 9_en-AU-Wavenet-A_0_14-1
276/290. Calculating PESQ scores for: 9_en-AU-Wavenet-B_0_10-0
277/290. Calculating PESQ scores for: 9_en-AU-Wavenet-B

In [14]:
results

{'0_en-AU-Wavenet-A_0_10-0': [2.833, 0.581, 0.935, 0.523, 0.922, 0.489],
 '0_en-AU-Wavenet-A_0_10-1': [3.158, 2.27, 2.487, 1.472, 2.602, 1.295],
 '0_en-AU-Wavenet-A_0_10-2': [2.718, 1.123, 1.694, 0.676, 1.618, 0.814],
 '0_en-AU-Wavenet-A_0_10-3': [2.783, 0.48, 0.529, 0.525, 0.301, 0.371],
 '0_en-AU-Wavenet-A_0_10-4': [2.824, 1.602, 2.003, 1.164, 2.013, 1.013],
 '0_en-AU-Wavenet-A_0_10-5': [3.221, 0.796, 0.777, 0.75, 0.604, 0.664],
 '0_en-AU-Wavenet-A_0_12-0': [2.656, 2.018, 1.942, 0.694, 1.947, 0.714],
 '0_en-AU-Wavenet-A_0_12-1': [2.748, 2.244, 2.336, 1.536, 2.669, 1.351],
 '0_en-AU-Wavenet-A_0_12-2': [2.849, 1.45, 1.092, 1.126, 1.193, 0.723],
 '0_en-AU-Wavenet-A_0_12-3': [2.842, 0.675, 0.634, 0.527, 0.484, 0.512],
 '0_en-AU-Wavenet-A_0_12-4': [2.83, 1.718, 1.549, 1.052, 1.727, 1.124],
 '0_en-AU-Wavenet-A_0_14-0': [2.776, 0.559, 0.648, 0.768, 0.648, 0.668],
 '0_en-AU-Wavenet-A_0_14-1': [2.964, 0.797, 0.762, 0.984, 0.841, 1.014],
 '0_en-AU-Wavenet-A_0_14-2': [2.763, 1.782, 1.715, 1.232

In [16]:
np.save(os.path.join(results_dir, "_results.npy"), results)

In [25]:
results_df = pd.DataFrame.from_dict(results, orient='index', columns=['clean_wav', 'noisy_wav', 
                                            'noisy_mel', 'denoised_mel', 
                                            'noisy_mel_clean_mel', 'denoised_mel_clean_mel' ])
results_df

,clean_wav,noisy_wav,noisy_mel,denoised_mel,noisy_mel_clean_mel,denoised_mel_clean_mel
0_en-AU-Wavenet-A_0_10-0,2.833,0.581,0.935,0.523,0.922,0.489
0_en-AU-Wavenet-A_0_10-1,3.158,2.270,2.487,1.472,2.602,1.295
0_en-AU-Wavenet-A_0_10-2,2.718,1.123,1.694,0.676,1.618,0.814
0_en-AU-Wavenet-A_0_10-3,2.783,0.480,0.529,0.525,0.301,0.371
0_en-AU-Wavenet-A_0_10-4,2.824,1.602,2.003,1.164,2.013,1.013
0_en-AU-Wavenet-A_0_10-5,3.221,0.796,0.777,0.750,0.604,0.664
0_en-AU-Wavenet-A_0_12-0,2.656,2.018,1.942,0.694,1.947,0.714
0_en-AU-Wavenet-A_0_12-1,2.748,2.244,2.336,1.536,2.669,1.351
0_en-AU-Wavenet-A_0_12-2,2.849,1.450,1.092,1.126,1.193,0.723
0_en-AU-Wavenet-A_0_12-3,2.842,0.675,0.634,0.527,0.484,0.512


In [30]:
results_df.mean()

clean_wav                 2.799869
noisy_wav                 1.452941
noisy_mel                 1.527148
denoised_mel              1.039590
noisy_mel_clean_mel       1.453028
denoised_mel_clean_mel    0.958117
dtype: float64

In [31]:
results_df.describe()

,clean_wav,noisy_wav,noisy_mel,denoised_mel,noisy_mel_clean_mel,denoised_mel_clean_mel
count,290.000000,290.000000,290.000000,290.000000,290.000000,290.000000
mean,2.799869,1.452941,1.527148,1.039590,1.453028,0.958117
std,0.216283,0.559106,0.532433,0.350235,0.594938,0.345022
min,1.687000,0.168000,0.131000,0.013000,0.186000,0.059000
25%,2.698500,1.057250,1.106750,0.796250,0.943000,0.722250
50%,2.802000,1.416500,1.545500,1.056500,1.464000,0.954000
75%,2.927250,1.904250,1.968500,1.301750,1.908500,1.202500
max,3.406000,2.674000,2.584000,2.154000,2.889000,2.063000
